<a href="https://colab.research.google.com/github/Yukiii0517/QM2/blob/main/w03_spatial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Panel Regression




## *Course Work*  [![Open In Colab](https://github.com/oballinger/QM2/blob/main/colab-badge.png?raw=1)](https://colab.research.google.com/github/oballinger/QM2/blob/main/notebooks/W03.%20Spatial%20Data.ipynb)


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import plotly.express as px
import warnings
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col
import matplotlib.pyplot as plt


In [3]:
warnings.filterwarnings('ignore')
sns.set(font_scale=1.5)
sns.set_style("white")
plt.rcParams['figure.figsize'] = (12, 8)

In [4]:
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf
from linearmodels.panel import compare

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 10.2 MB/s eta 0:00:00


In [1]:
from google.colab import files
uploaded = files.upload()

Saving QM2_Date.csv to QM2_Date.csv


In [6]:
df = pd.read_csv('QM2_Date.csv')
df.head()

,State,Year,Real per capita total gambling expenditure value ($),Wage price index,Male Unemployment rate (%),Female Unemployment rate (%),Final consumption expenditure minus net loss from gambling (Millions $),Unnamed: 7
0,ACT,1998–99,1739.17,69.25,6.79,5.10,11739.0,NaN
1,ACT,1999–00,1757.75,70.98,5.63,4.38,12528.0,NaN
2,ACT,2000–01,1761.04,73.50,5.26,3.83,12807.0,NaN
3,ACT,2001–02,1718.74,75.80,5.25,3.72,13462.0,NaN
4,ACT,2002–03,1723.14,78.40,4.80,3.67,14253.0,NaN


In [7]:
df.dropna(subset=['Year'], inplace=True)
df['year'] = df['Year'].str[-2:].astype(int) + 2000
df.loc[df['year'] > 2025, 'year'] -= 100  # fixes 1998–99
df

,State,Year,Real per capita total gambling expenditure value ($),Wage price index,Male Unemployment rate (%),Female Unemployment rate (%),Final consumption expenditure minus net loss from gambling (Millions $),Unnamed: 7,year
0,ACT,1998–99,1739.17,69.25,6.79,5.10,11739.0,NaN,1999
1,ACT,1999–00,1757.75,70.98,5.63,4.38,12528.0,NaN,2000
2,ACT,2000–01,1761.04,73.50,5.26,3.83,12807.0,NaN,2001
3,ACT,2001–02,1718.74,75.80,5.25,3.72,13462.0,NaN,2002
4,ACT,2002–03,1723.14,78.40,4.80,3.67,14253.0,NaN,2003
...,...,...,...,...,...,...,...,...,...
203,WA,2019–20,787.04,132.48,5.98,6.07,126209.0,NaN,2020
204,WA,2020–21,971.28,134.43,6.19,5.97,129351.0,NaN,2021
205,WA,2021–22,946.83,137.35,3.86,3.74,137505.0,NaN,2022
206,WA,2022–23,925.11,142.58,3.66,3.44,144880.0,NaN,2023


In [10]:
# Create unemployment rate
df['unemp'] = (df['UNEMPLOYMENT RATE (male)'] +
               df['UNEMPLOYMENT RATE (female)']) / 2

# Logs
df['log_gambling'] = np.log(df['REAL PER CAPITA TOTAL GAMBLING EXPENDITURE Value ($) '])
df['log_wpi'] = np.log(df['WAGE PRICE INDEX'])
df['log_cons'] = np.log(df['FINAL CONSUMPTION EXPENDITURE MINUS NET LOSS FROM GAMBLING'])

# Lags
df['log_gambling_l1'] = df['log_gambling'].shift(1)
df['unemp_l1'] = df['unemp'].shift(1)

KeyError: 'UNEMPLOYMENT RATE (male)'

In [84]:
import statsmodels.api as sm
X_shock = sm.add_constant(df[['log_gambling_l1', 'log_wpi', 'log_cons']].dropna())
y_shock = df.loc[X_shock.index, 'unemp']

model_shock = sm.OLS(y_shock, X_shock).fit(cov_type='HC1')
print(model_shock.summary())


                            OLS Regression Results                            
Dep. Variable:                  unemp   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     8.180
Date:                Mon, 22 Dec 2025   Prob (F-statistic):           0.000850
Time:                        08:48:12   Log-Likelihood:                -13.641
No. Observations:                  25   AIC:                             35.28
Df Residuals:                      21   BIC:                             40.16
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             117.9441     47.741     

In [13]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

def _end_year_safe(x):
    if pd.isna(x):
        return None
    x = str(x).strip()
    start = int(x[:4])
    end2 = int(x[-2:])
    end = (start // 100) * 100 + end2
    if end < start:  # handles 1999–00 -> 2000
        end += 100
    return end


def prepare_region_df(
    df: pd.DataFrame,
    *,
    col_year: str = "Year",
    col_gambling: str = "Real per capita total gambling expenditure value ($)", # Corrected
    col_wpi: str = "Wage price index", # Corrected
    col_unemp_m: str = "Male Unemployment rate (%)", # Corrected
    col_unemp_f: str = "Female Unemployment rate (%)", # Corrected
    col_cons: str = "Final consumption expenditure minus net loss from gambling (Millions $)", # Corrected
) -> pd.DataFrame:
    out = df.copy()

    out["year"] = out[col_year].apply(_end_year_safe)
    out = out.dropna(subset=["year"]).sort_values("year").reset_index(drop=True)

    out["gambling"] = out[col_gambling]
    out["wpi"] = out[col_wpi]
    out["cons"] = out[col_cons]
    out["unemp"] = (out[col_unemp_m] + out[col_unemp_f]) / 2

    return out

In [42]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

# ==========================================================
# SHOCK MODELS (two equations, consumption as control)
# ==========================================================
def run_modelB2_shock(df, lags=(1, 2, 3, 4, 5, 6, 7, 8), robust_se="HC1"):
    """
    Shock models:
    (1) unemp_t = α + β_k gambling_{t-k} + δ cons_t + ε_t
    (2) wpi_t   = α + β_k gambling_{t-k} + δ cons_t + η_t
    """

    df = df.copy()

    # create lagged gambling once
    for k in lags:
        df[f"gambling_l{k}"] = df["gambling"].shift(k)

    # ---------- Equation (1): unemployment ----------
    models_u, names = [], []
    for k in lags:
        data = df.dropna(subset=["unemp", f"gambling_l{k}", "cons"])
        m = ols(
            formula=f"unemp ~ gambling_l{k} + cons",
            data=data
        ).fit(cov_type=robust_se)

        models_u.append(m)
        names.append(f"k={k}")

    table_u = summary_col(
        models_u,
        stars=True,
        float_format="%0.3f",
        model_names=names,
        info_dict={"N": lambda x: f"{int(x.nobs)}"}
    )

    # ---------- Equation (2): WPI ----------
    models_w = []
    for k in lags:
        data = df.dropna(subset=["wpi", f"gambling_l{k}", "cons"])
        m = ols(
            formula=f"wpi ~ gambling_l{k} + cons",
            data=data
        ).fit(cov_type=robust_se)

        models_w.append(m)

    table_w = summary_col(
        models_w,
        stars=True,
        float_format="%0.3f",
        model_names=names,
        info_dict={"N": lambda x: f"{int(x.nobs)}"}
    )

    return (models_u, models_w), (table_u, table_w)


In [43]:
# ==========================================================
# SIGNAL MODEL (consumption as control)
# ==========================================================
def run_modelB2_signal(df, lags=(1, 2, 3, 4, 5, 6, 7, 8), robust_se="HC1"):
    """
    Signal model:
    gambling_t = α + γ_k unemp_{t-k} + φ_k wpi_{t-k} + θ cons_{t-k} + ν_t
    """

    df = df.copy()

    models, names = [], []

    for k in lags:
        df[f"unemp_l{k}"] = df["unemp"].shift(k)
        df[f"wpi_l{k}"] = df["wpi"].shift(k)
        df[f"cons_l{k}"] = df["cons"].shift(k)

        data = df.dropna(
            subset=["gambling", f"unemp_l{k}", f"wpi_l{k}", f"cons_l{k}"]
        )

        m = ols(
            formula=f"gambling ~ unemp_l{k} + wpi_l{k} + cons_l{k}",
            data=data
        ).fit(cov_type=robust_se)

        models.append(m)
        names.append(f"k={k}")

    table = summary_col(
        models,
        stars=True,
        float_format="%0.3f",
        model_names=names,
        info_dict={"N": lambda x: f"{int(x.nobs)}"}
    )

    return models, table


In [54]:
df_all= df.copy()
df_act = df_all[df_all['State'] == 'ACT'] # Corrected filtering syntax
df_prepared = prepare_region_df(df_all)
# Prepare the DataFrame using the defined function
(shock_u_models, shock_w_models), (shock_u_table, shock_w_table) = run_modelB2_shock(
    df_prepared, lags=(1, 2, 3, 4, 5, 6, 7, 8)
)

print("Shock: Unemployment equation")
print(shock_u_table)

print("\nShock: WPI equation")
print(shock_w_table)

signal_models, signal_table = run_modelB2_signal(
    df_prepared, lags=(1, 2, 3, 4, 5, 6, 7, 8)
)

print("\nSignal equation")
print(signal_table)

Shock: Unemployment equation

                 k=1      k=2      k=3      k=4      k=5      k=6      k=7      k=8   
--------------------------------------------------------------------------------------
Intercept      5.277*** 5.378*** 5.336*** 5.223*** 5.412*** 5.275*** 5.264*** 5.158***
               (0.154)  (0.162)  (0.160)  (0.161)  (0.180)  (0.156)  (0.165)  (0.152) 
gambling_l1    0.000**                                                                
               (0.000)                                                                
cons           -0.000   -0.000   -0.000   -0.000   -0.000   -0.000   -0.000   0.000   
               (0.000)  (0.000)  (0.000)  (0.000)  (0.000)  (0.000)  (0.000)  (0.000) 
gambling_l2             -0.000                                                        
                        (0.000)                                                       
gambling_l3                      0.000                                                
             

In [51]:
df_prepared

,State,Year,Real per capita total gambling expenditure value ($),Wage price index,Male Unemployment rate (%),Female Unemployment rate (%),Final consumption expenditure minus net loss from gambling (Millions $),Unnamed: 7,year,gambling,wpi,cons,unemp
0,ACT,1998–99,1739.17,69.25,6.79,5.10,11739.0,NaN,1999,1739.17,69.25,11739.0,5.945
1,NSW,1998–99,2164.89,69.88,6.68,6.53,225975.0,NaN,1999,2164.89,69.88,225975.0,6.605
2,NT,1998–99,1838.10,70.85,4.11,3.34,6530.0,NaN,1999,1838.10,70.85,6530.0,3.725
3,QLD,1998–99,1585.85,69.43,8.15,7.76,108897.0,NaN,1999,1585.85,69.43,108897.0,7.955
4,SA,1998–99,1283.63,69.58,9.32,8.24,46333.0,NaN,1999,1283.63,69.58,46333.0,8.780
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NSW,2023-24,2007.01,148.95,3.75,3.59,435652.0,NaN,2024,2007.01,148.95,435652.0,3.670
204,VIC,2023-24,1357.21,150.25,4.08,3.88,340396.0,NaN,2024,1357.21,150.25,340396.0,3.980
205,TAS,2023-24,887.73,153.30,4.51,3.73,26392.0,NaN,2024,887.73,153.30,26392.0,4.120
206,SA,2023-24,1200.28,149.85,3.94,3.75,87826.0,NaN,2024,1200.28,149.85,87826.0,3.845


In [59]:
def run_modelB_shock(
    df_region: pd.DataFrame,
    *,
    lags=(1, 2, 3, 4, 5),
    robust_se: str = "HC1"
):

    df = df_region.copy()

    models = []
    names = []

    for k in lags:
        df[f"gambling_l{k}"] = df["gambling"].shift(k)
        data = df.dropna(subset=["unemp", f"gambling_l{k}", "wpi", "cons"])

        m = ols(
            formula=f"unemp ~ gambling_l{k} + wpi + cons",
            data=data
        ).fit(cov_type=robust_se)

        models.append(m)
        names.append(f"k={k}")

    table = summary_col(
        models,
        stars=True,
        float_format="%0.3f",
        model_names=names,
        info_dict={"N": lambda x: f"{int(x.nobs)}"}
    )

    return models, table


def run_modelB_signal(
    df_region: pd.DataFrame,
    *,
    lags=(1, 2, 3, 4, 5),
    robust_se: str = "HC1"
):

    df = df_region.copy()

    models = []
    names = []

    for k in lags:
        df[f"unemp_l{k}"] = df["unemp"].shift(k)
        df[f"wpi_l{k}"] = df["wpi"].shift(k)
        df[f"cons_l{k}"] = df["cons"].shift(k)

        data = df.dropna(subset=["gambling", f"unemp_l{k}", f"wpi_l{k}", f"cons_l{k}"])

        m = ols(
            formula=f"gambling ~ unemp_l{k} + wpi_l{k} + cons_l{k}",
            data=data
        ).fit(cov_type=robust_se)

        models.append(m)
        names.append(f"k={k}")

    table = summary_col(
        models,
        stars=True,
        float_format="%0.3f",
        model_names=names,
        info_dict={"N": lambda x: f"{int(x.nobs)}"}
    )

    return models, table

In [61]:
df_all= df.copy()
df_act = df_all[df_all['State'] == 'ACT'] # Corrected filtering syntax
# Prepare the DataFrame using the defined function
df_prepared = prepare_region_df(df_act)

shock_models, shock_table = run_modelB_shock(df_prepared, lags=(1,2,3,4,5))
print(shock_table)

signal_models, signal_table = run_modelB_signal(df_prepared, lags=(1,2,3,4,5))
print(signal_table)


                 k=1      k=2      k=3      k=4       k=5   
------------------------------------------------------------
Intercept      6.330*** 8.297*** 9.716** 13.926*** 12.394***
               (1.438)  (2.487)  (3.997) (3.794)   (3.891)  
gambling_l1    -0.001                                       
               (0.001)                                      
wpi            0.067    0.026    0.029   0.028     0.067**  
               (0.043)  (0.044)  (0.039) (0.030)   (0.028)  
cons           -0.000*  -0.000   -0.000* -0.000**  -0.001***
               (0.000)  (0.000)  (0.000) (0.000)   (0.000)  
gambling_l2             -0.002*                             
                        (0.001)                             
gambling_l3                      -0.002                     
                                 (0.001)                    
gambling_l4                              -0.003***          
                                         (0.001)            
gambling_l5            